# Kaggle-Freesound-audio-tagging

https://www.kaggle.com/c/freesound-audio-tagging

Решение дает 0.8 очков на Public Leaderboard, Ваша задача преодолеть 0.9.


Проверьте, что папка с данными лежит по пути `DATADIR`, архивы распакованы.
Потестировано на версиях python 2.7/3.5/3.6, tf>=1.4. 

Код под одну карточку, так что на машинах с несколькими запускайте с ограничениями `CUDA_VISIBLE_DEVICES=x jupyter notebook ...` или модифицируйте код.

Более гибкое решение можно описать в файлах (см. README.md)


In [ ]:
from __future__ import print_function
import tensorflow as tf
from tensorflow.contrib import layers
from sklearn.model_selection import train_test_split
from tensorflow.contrib.learn.python.learn.learn_io.generator_io import generator_input_fn
import utils
import pandas as pd
import numpy as np
import os
import json

# todo: поправить путь
DATADIR='~/data/kaggle-freesound/'
OUTDIR = './runs/'

In [ ]:
# опишем сетку -- feature extractor, на вход она получит _картинку_, 
# тензор с размерами [?, ?, ?, 2], на выходе выдаст логиты [?, num_classes]
# постарайтесь все необходимые параметры передать через params
def baseline(x, params, is_training):
    # это общие параметры для сверточных слоев, мы будем передавать их явно:
    afn = dict(
        normalizer_fn=layers.batch_norm,
        normalizer_params=dict(is_training=is_training),
    )

    for i in range(3):
        if is_training:
            x = tf.nn.dropout(x, 0.9)
        x = layers.conv2d(x, 16 * (2 ** i), (3, 11), **afn)
        x = layers.max_pool2d(x, 2, 2)

    gap = tf.reduce_mean(x, axis=[1, 2], keep_dims=True)
    gap = tf.nn.dropout(gap, params.keep_prob if is_training else 1.0)

    # вместо полносвязного слоя удобно взять свертку 1х1 на нужное количество классов
    x = tf.layers.conv2d(gap, params.num_classes, 1, activation=None)

    return tf.squeeze(x, [1, 2])


# Опишем модельку:
#   features -- словарь с входными тензорами, 
#   labels -- тензор с метками, 
#   mode -- один из трех вариантов tf.estimator.ModeKeys.TRAIN/EVAL/PREDICT
#   params -- набор параметров, который мы сформируем позже
#
#   функция должна вернуть правильно заполненный tf.estimator.EstimatorSpec(**specs), 
#     см документацию и комменты в исходном коде, если хотите разобраться глубже

def model_handler(features, labels, mode, params, config):
    # todo: добавьте сюда выбор модельки по параметру из params
    extractor = tf.make_template(
        'extractor', baseline,
        create_scope_now_=True,
    )

    wav = features['wav']  # здесь будет тензор [bs, timesteps]
    specgram = signal.stft(wav, 400, 160)  # здесь комплекснозначный тензор [bs, time_bins, freq_bins]

    phase = tf.angle(specgram) / np.pi
    amp = tf.log1p(tf.abs(specgram))

    x = tf.stack([amp, phase], axis=3)  # здесь почти обычная картинка  [bs, time_bins, freq_bins, 2]
    x = tf.to_float(x)

    logits = extractor(x, params, mode == tf.estimator.ModeKeys.TRAIN)
    predictions = tf.nn.softmax(logits)

    if mode == tf.estimator.ModeKeys.TRAIN:
        loss = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                labels=labels, logits=logits)
        )

        # todo: обязательно попробуйте другие варианты изменния lr
        def _learning_rate_decay_fn(learning_rate, global_step):
            return tf.train.exponential_decay(
                learning_rate, global_step, decay_steps=10000, decay_rate=0.99)

        train_op = tf.contrib.layers.optimize_loss(
            loss=loss,
            global_step=tf.contrib.framework.get_global_step(),
            learning_rate=params.learning_rate,
            optimizer=lambda lr: tf.train.AdamOptimizer(lr),  # оптимизатор точно стоит потюнить
            learning_rate_decay_fn=_learning_rate_decay_fn,
            clip_gradients=params.clip_gradients,
            variables=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES))

        specs = dict(
            mode=mode,
            loss=loss,
            train_op=train_op,
        )

    if mode == tf.estimator.ModeKeys.EVAL:
        acc, acc_op = tf.metrics.accuracy(labels, tf.argmax(logits, axis=-1))
        # см https://www.kaggle.com/c/freesound-audio-tagging#evaluation
        # метрика оценки mean average precision at 3 (MAP@3)
        # нужно чтобы среди трех топовых предсказаний была правильная метка
        map3, map3_op = tf.metrics.sparse_average_precision_at_k(
            tf.cast(labels, tf.int64), predictions, 3)
        loss = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                labels=labels, logits=logits)
        )
        specs = dict(
            mode=mode,
            loss=loss,
            eval_metric_ops={
                "MAP@1": (acc, acc_op),
                "MAP@3": (map3, map3_op),
            }
        )

    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            # здесь можно пробрасывать что угодно
            'predictions': predictions,  # весь вектор предсказаний
            'top3': tf.nn.top_k(predictions, 3)[1],  # топ-3 метки
            'prediction': tf.argmax(predictions, 1),  # топовая метка
            'fname': features['fname'],  # имя файла, удобный ход
        }
        specs = dict(
            mode=mode,
            predictions=predictions,
        )
    return tf.estimator.EstimatorSpec(**specs)

In [ ]:
# создадим очередную папку для модельки
outdir = utils.get_new_model_path(OUTDIR)

# собираем вместе параметры
params = dict(
    outdir=outdir,
    seed=2018,
    datadir=DATADIR,
    batch_size=32,
    keep_prob=0.8,
    learning_rage=0.8,
    clip_gradients=15.0,
    # todo: придумать, что с классами
    num_classes=len(label2id),
)


hparams = tf.contrib.training.HParams(**params)

with open(os.path.join(outdir, 'hparams.json'), 'w') as fout:
    json.dump(params, fout, indent=2)
with open(os.path.join(outdir, 'vocab.json'), 'w') as fout:
    json.dump(id2label, fout, indent=2)



# последний этап перед запуском -- организовать входные данные
# просто читаем и разбиваем на train/val
df = pd.read_csv(os.path.join(DATADIR, 'train.csv'))
labels = sorted(set(df.label.values))

label2id = {label: i for i, label in enumerate(labels)}
id2label = {i: label for label, i in label2id.items()}

df['label'] = [label2id[_] for _ in df.label.values]
df['fname'] = [
    os.path.join(DATADIR, 'audio_train', _) for _ in df.fname.values]

# todo: разберитесь с форматом входных данных, потюньте процедуру разбиения
# можно добавить фолды, балансировать классы или разбивать по флагу ручной разметки
idx = np.arange(len(df))
idx_train, idx_val = train_test_split(
    idx, test_size=0.33, random_state=2018, shuffle=True)
df_train, df_val = df.iloc[idx_train], df.iloc[idx_val]

# делаем генератры данных с предварительным чтением всех файлов в память
train_input_fn = generator_input_fn(
    x=utils.fast_datagenerator(df_train, hparams, 'train'),
    target_key='target',
    batch_size=hparams.batch_size,
    shuffle=True,
    queue_capacity=3 * hparams.batch_size,
    num_threads=1,
)

val_input_fn = generator_input_fn(
    x=utils.fast_datagenerator(df_val, hparams, 'val'),
    target_key='target',
    batch_size=hparams.batch_size,
    shuffle=False,
    queue_capacity=3 * hparams.batch_size,
    num_threads=1,
)

In [ ]:
run_config = tf.estimator.RunConfig(model_dir=hparams.outdir)


est = tf.estimator.Estimator(
    model_fn=model_handler,
    config=config,
    params=hparams,
)

train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn)
eval_spec = tf.estimator.EvalSpec(input_fn=val_input_fn)

tf.estimator.train_and_evaluate(est, train_spec, eval_spec)

Запустите tensorboard и смотрите графики обучения

```
tensorboard --logdir=./OUTDIR/   # в терминале на сервере
ssh -L 7007:localhost:6006 # в терминале у себя на машине
```
Зайдите браузером на http://localhost:6006


Теперь делаем предсказание в modeldir надо указать папку с желаемой моделью


In [ ]:
modeldir = ...
# сделать предсказание
with open(os.path.join(args.modeldir, 'hparams.json'), 'r') as fin:
    params = json.load(fin)

with open(os.path.join(args.modeldir, 'vocab.json'), 'r') as fin:
    vocab = json.load(fin)
    vocab = {int(k): v for k, v in vocab.items()}
    
params['model_dir'] = modeldir
    
hparams = tf.contrib.training.HParams(**params)

run_config = tf.estimator.RunConfig(
    model_dir=modeldir, session_config=session_config)

# создаем модельку
model = base.create_model(config=run_config, hparams=hparams)

# готовим данные для теста из sample_submission
df = pd.read_csv(os.path.join(args.datadir, 'sample_submission.csv'))
df.label = 0
df.fname = [
    os.path.join(args.datadir, 'audio_test', _)
    for _ in df.fname.values]

# predict все равно работает по одному примеру, так что давайте уберем батчи
# так мы сможем работать с записями целиком
# NB: стоит проверить, правильно ли работает pad_value
test_input_fn = generator_input_fn(
    x=utils.fast_datagenerator(df, params, 'test'),
    batch_size=1,
    shuffle=False,
    num_epochs=1,
    queue_capacity=hparams.batch_size,
    num_threads=1,
    pad_value=0.0,
)

it = model.predict(input_fn=test_input_fn)  # это итератор

# далее немного грязно, отрефакторите, добавьте информацию о фолдах, если нужно
submission = dict()
for output in tqdm(it):
    path = output['fname'].decode()
    fname = os.path.basename(path)
    # допускается предсказывать три метки на каждую запись
    predicted = " ".join([vocab[i] for i in output['top3']])
    submission[fname] = predicted

with open(os.path.join(args.modeldir, 'submission.csv'), 'w') as fout:
    fout.write('fname,label\n')
    for fname, pred in submission.items():
        fout.write("{},{}\n".format(fname, pred))